In [62]:
import pandas as pd

In [63]:
path_for_work = r"C:\Users\User\Desktop"

In [64]:
name = r"\Sell_Out_Екатеринбург_Уралтабак" +"_SKU.xlsx"

In [65]:
osh = pd.read_excel(r"D:\Analysis Burn\Дистрибьютеры отчетность\Source\Отчеты\Екатеринбург Урал табак\Sell_Out_Екатеринбург_Уралтабак.xlsx")

In [66]:
osh.head(1)

,Торговый представитель,Код клиента,Сегмент,Клиент,Адрес доставки,Дата отгрузки,Бренд,Группа,Вкус,Граммовка,Количество шт,ВЕС,Цена,стоимость,Дистрибьютор
0,не указан,УТ0000697,Спец. магазин,Абдуллин Руслан Илхатович,"622016, Свердловская обл, Нижний Тагил г, Косм...",2023-03-01,Overdose,Overdose 25 гр,Melon Berry (Ягодная дыня),25,1,0.025,205.0,205.0,2


In [67]:
osh['Дата отгрузки'].dtypes == '<M8[ns]'

True

In [68]:
if osh['Дата отгрузки'].dtypes != '<M8[ns]':
    if osh['Дата отгрузки'].astype(str).str.match("\d{4}-\d{2}-\d{2}").all():
        osh['Дата отгрузки'] = pd.to_datetime(osh['Дата отгрузки'], yearfirst = True)
    elif osh['Дата отгрузки'].astype(str).str.match("\d{2}-\d{2}-\d{4}").all():
        osh['Дата отгрузки'] = pd.to_datetime(osh['Дата отгрузки'], dayfirst = True)
    else:
        show_notification("С датой что-то не так, проверьте плз")
        exit()

In [69]:
osh["Год"] = osh["Дата отгрузки"].dt.year
osh["Месяц"] = osh["Дата отгрузки"].dt.month

In [70]:
osh["Месяц"].unique()

array([3, 4, 5, 6], dtype=int64)

In [71]:
osh["Год"].unique()

array([2023], dtype=int64)

In [72]:
# пять месяцев отфильтровать
# osh = osh.loc[(osh['Год'] == 2023)]

In [73]:
osh_2 = osh.copy(deep=True)

In [74]:
osh_2.loc[:, 'Месяц название'] = osh_2.loc[:,'Год'].astype(str) + '-' + osh_2.loc[:, 'Месяц'].astype(str)

In [75]:
pivot_table = osh_2.pivot_table(values='ВЕС',
                             index=['Дистрибьютор','Клиент', 'Бренд', 'Вкус', 'Группа'],
                             columns='Месяц название',
                             aggfunc=lambda x: 1 if sum(x) > 0 else 0,
                             fill_value=0).reset_index()

In [76]:
# сегмент только для ошишы
# 
# pivot_table = osh_2.pivot_table(values='ВЕС',
#                              index=['Дистрибьютор','Сегмент','Клиент', 'Бренд', 'Вкус'],
#                              columns='Месяц название',
#                              aggfunc=lambda x: 1 if sum(x) > 0 else 0,
#                              fill_value=0).reset_index()

In [77]:
# здесь править цифру в iloc
pivot_table['Общая сумма'] = pivot_table.iloc[:, 5:].sum(axis=1)

In [78]:
# pivot_table.columns = pivot_table.columns.map(lambda x: '_'.join(map(str, x)))

In [79]:
pivot_table

Месяц название,Дистрибьютор,Клиент,Бренд,Вкус,Группа,2023-3,2023-4,2023-5,2023-6,Общая сумма
0,2,АВРОРА ООО,Blackburn,Ananas Shock (Кислый ананас),BlackBurn 25 гр,0,1,1,0,2
1,2,АВРОРА ООО,Blackburn,Ananas Shock (Кислый ананас),Blackburn 25 гр,0,0,0,1,1
2,2,АВРОРА ООО,Blackburn,Barberry Shock (Кислый барбарис),BlackBurn 25 гр,0,1,1,0,2
3,2,АВРОРА ООО,Blackburn,Barberry Shock (Кислый барбарис),Blackburn 25 гр,0,0,0,1,1
4,2,АВРОРА ООО,Blackburn,ICE BABY (Ягодный сорбет с грейпфрутом) (COLLA...,BlackBurn 25 гр,0,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...
17080,2,япона Пышма,Blackburn,Pear lemonade (Something Fresh) - Грушевый лим...,BlackBurn 200 гр,0,1,0,0,1
17081,2,япона Пышма,Blackburn,Raspberries (Малина),BlackBurn 200 гр,0,1,0,0,1
17082,2,япона Пышма,Blackburn,Something Berry (Что-то ягодное),BlackBurn 200 гр,0,1,0,0,1
17083,2,япона Пышма,Blackburn,Something Berry (Что-то ягодное),Blackburn 200 гр,0,0,0,1,1


In [80]:
pivot_table.loc[pivot_table['Общая сумма'] == 0]

Месяц название,Дистрибьютор,Клиент,Бренд,Вкус,Группа,2023-3,2023-4,2023-5,2023-6,Общая сумма
2003,2,Васин Иван Алексеевич,Overdose,"Brumblebee (Клубника, ежевика, голубика)",Overdose 200 гр,0,0,0,0,0
2007,2,Васин Иван Алексеевич,Overdose,Fantola (Тропическая газировка),Overdose 200 гр,0,0,0,0,0
2011,2,Васин Иван Алексеевич,Overdose,Frustyle (Кактус-лайм),Overdose 200 гр,0,0,0,0,0
2020,2,Васин Иван Алексеевич,Overdose,Manila Malina (Филиппинская малина),Overdose 200 гр,0,0,0,0,0
2033,2,Васин Иван Алексеевич,Overdose,Watermelon (Сахарный арбуз),Overdose 200 гр,0,0,0,0,0
2970,2,Гребенников Александр Леонидович,Overdose,Fantola (Тропическая газировка),Overdose 200 гр,0,0,0,0,0
2973,2,Гребенников Александр Леонидович,Overdose,Frustyle (Кактус-лайм),Overdose 200 гр,0,0,0,0,0
3686,2,Евдокимов Егор Владиславович,PeterRalf,Banana Сrepes (Утренний завтрак с бананом),PeterRalf 50 гр,0,0,0,0,0
4666,2,Замараева Светлана Евгеньевна,Blackburn,Lemon Shock (Кислый лимон),Blackburn 200 гр,0,0,0,0,0
6446,2,Козлова Анна Сергеевна,Blackburn,BlackBerry Lemonade (Ежевичный лимонад),Blackburn 200 гр,0,0,0,0,0


In [81]:
pivot_table['Общая сумма'].unique()

array([2, 1, 3, 4, 0], dtype=int64)

In [82]:
pivot_table3 = osh_2.pivot_table(values='ВЕС',
                             index=['Дистрибьютор','Клиент', 'Бренд', 'Вкус'],
                             columns='Месяц название',
                             aggfunc=lambda x: 1 if sum(x) > 0 else 0,
                             fill_value=0).reset_index()

In [83]:
pivot_table3

Месяц название,Дистрибьютор,Клиент,Бренд,Вкус,2023-3,2023-4,2023-5,2023-6
0,2,АВРОРА ООО,Blackburn,Ananas Shock (Кислый ананас),0,1,1,1
1,2,АВРОРА ООО,Blackburn,Barberry Shock (Кислый барбарис),0,1,1,1
2,2,АВРОРА ООО,Blackburn,ICE BABY (Ягодный сорбет с грейпфрутом) (COLLA...,0,1,0,1
3,2,АВРОРА ООО,Blackburn,Melon Halls (Дынный холс),0,1,1,1
4,2,АВРОРА ООО,Blackburn,Papaya v obed (Папайя) (COLLAB KMTM),0,1,1,1
...,...,...,...,...,...,...,...,...
11798,2,япона Пышма,Blackburn,Lemon Shock (Кислый лимон),0,1,0,1
11799,2,япона Пышма,Blackburn,Pear lemonade (Something Fresh) - Грушевый лим...,0,1,0,0
11800,2,япона Пышма,Blackburn,Raspberries (Малина),0,1,0,0
11801,2,япона Пышма,Blackburn,Something Berry (Что-то ягодное),0,1,0,1


In [84]:
# здесь править цифру в iloc
pivot_table3['Общая сумма'] = pivot_table3.iloc[:, 4:].sum(axis=1)

In [85]:
pivot_table2 = pivot_table3.pivot_table(values='Клиент',
                             index=['Дистрибьютор','Бренд', 'Вкус'],
                             columns='Общая сумма',
                             aggfunc=pd.Series.count,
                             fill_value=0,
                             margins=False).reset_index()

In [86]:
pivot_table2

Общая сумма,Дистрибьютор,Бренд,Вкус,0,1,2,3,4
0,2,B3,Orange Cream (Кремовый апельсин),0,1,0,0,0
1,2,Banger,BBQ (Соус Барбекю),0,10,5,0,0
2,2,Banger,Batumi (Ягоды с гранатом и маскарпоне),0,23,6,2,0
3,2,Banger,Berry Pie (Малиновый пирог),0,23,5,0,0
4,2,Banger,"Black and White (Печенье ""Oreo"")",0,24,3,1,0
...,...,...,...,...,...,...,...,...
152,2,PeterRalf,Post Melon (Дынная жевательная резинка),0,2,0,0,1
153,2,PeterRalf,Quince (Травянистый вкус айвы),0,4,0,0,0
154,2,PeterRalf,Santa-Barbara (Барбарисовый леденец),0,1,0,0,0
155,2,PeterRalf,"Sugar Baby (Арбуз, мята и лед)",0,3,0,1,0


In [64]:
# сегмент только ошише нужен
# pivot_table2 = pivot_table.pivot_table(values='Клиент',
#                              index=['Сегмент','Бренд', 'Вкус'],
#                              columns='Общая сумма',
#                              aggfunc=pd.Series.count,
#                              fill_value=0,
#                              margins=False).reset_index()

In [87]:
pivot_table.columns.values

array(['Дистрибьютор', 'Клиент', 'Бренд', 'Вкус', 'Группа', '2023-3',
       '2023-4', '2023-5', '2023-6', 'Общая сумма'], dtype=object)

In [95]:
partners = pd.read_excel(r"D:\Analysis Burn\Дистрибьюторы Burn.xlsx", sheet_name='Дистрибьюторы')

In [94]:
#
#  правильное расположение колонок у первой таблицы
# 
columns = pivot_table.columns.values

pivot_table['Клиент_2'] = pd.NaT

# Добавление колонки 'Клиент1'
columns = ['Дистрибьютор'] + ['Клиент_2'] + [col for col in columns if col != 'Клиент_2' and col != 'Дистрибьютор']

pivot_table = pivot_table[columns]

pivot_table = pivot_table.merge(partners[['Город', 'Партнер ', 'Уникальный идентификатор']], how='left', left_on= 'Дистрибьютор', right_on='Уникальный идентификатор')

columns = ['Город'] + ['Партнер'] + [col for col in columns]

pivot_table = pivot_table[columns]

# 
# правильное расположение колонок у второй таблицы
# 
columns2 = pivot_table2.columns.values

pivot_table2 = pivot_table2.merge(partners[['Город', 'Партнер ', 'Уникальный идентификатор']], how='left', left_on= 'Дистрибьютор', right_on='Уникальный идентификатор')

columns2 = ['Город'] + ['Партнер'] + [col for col in columns2]
pivot_table2 = pivot_table2[columns2]

['Дистрибьютор',
 'Клиент_2',
 'Клиент',
 'Бренд',
 'Вкус',
 'Группа',
 '2023-3',
 '2023-4',
 '2023-5',
 '2023-6',
 'Общая сумма']

In [83]:
with pd.ExcelWriter(path_for_work + name) as writer:  
    pivot_table2.to_excel(writer, sheet_name='1', index=False)  
    pivot_table.to_excel(writer, sheet_name='2', index=False)